In [1]:
import pandas as pd

def read_file(filepath: str):
    return pd.read_json(filepath, orient="records")

df = read_file("obfuscated_data_06.json")

In [2]:
# import json

# Assuming df is your DataFrame that contains the 'tokens' column
total_tokens = sum(len(df.iloc[i].tokens) for i in range(len(df)))

print(f"Total number of tokens: {total_tokens}")

Total number of tokens: 16746176


In [7]:
# Cost per token in dollars (GPT-4o)
input_cost_per_token_gpt4o = 5 / 1000000  # Cost per input token
output_cost_per_token_gpt4o = 15 / 1000000  # Cost per output token

# Cost per token in dollars (GPT-4o-mini)
input_cost_per_token_gpt4omini = 0.15 / 1000000  # Cost per input token
output_cost_per_token_gpt4omini = 0.6 / 1000000  # Cost per output token

# Calculate the total cost
total_cost_gpt4o = total_tokens * (input_cost_per_token_gpt4o + output_cost_per_token_gpt4o)
total_cost_gpt4omini = total_tokens * (input_cost_per_token_gpt4omini + output_cost_per_token_gpt4omini)

print(f"Estimated total cost (GPT-4o): ${total_cost_gpt4o:.2f}")
print(f"Estimated total cost (GPT-4o-mini): ${total_cost_gpt4omini:.2f}")

Estimated total cost (GPT-4o): $334.92
Estimated total cost (GPT-4o-mini): $12.56


**Consider Azure AI as baseline**
Model performance first
Efficiency (average time to process one transcript and show the length of each transcript),
cost ()